In [1]:
import sys
sys.path.append('..')
from luriegold import luriegold
from illmat import illmat
import numpy as np

## Example - luriegold

First, create a random ill-conditioned correlation matrix for three variables.

In [2]:
d = 4
R = illmat(d, random_state=2)
R.round(3)

array([[ 1.   , -0.948,  0.099, -0.129],
       [-0.948,  1.   , -0.591,  0.239],
       [ 0.099, -0.591,  1.   ,  0.058],
       [-0.129,  0.239,  0.058,  1.   ]])

Second, try to compute a feasible solution for $x$

In [3]:
C, L, res = luriegold(R)
print(C.round(3))

[[ 1.    -0.885  0.128 -0.138]
 [-0.885  1.    -0.555  0.227]
 [ 0.128 -0.555  1.     0.053]
 [-0.138  0.227  0.053  1.   ]]


As a last check try to create the Cholesky matrix from $C$.
If it does not work, numpy would throw an exception.

In [4]:
np.linalg.cholesky(C).round(3)

array([[ 1.   ,  0.   ,  0.   ,  0.   ],
       [-0.885,  0.466,  0.   ,  0.   ],
       [ 0.128, -0.947,  0.295,  0.   ],
       [-0.138,  0.226,  0.964,  0.001]])

In [5]:
print("Is Det(C)>=0 ? ", np.linalg.det(C))

Is Det(C)>=0 ?  1.1028467112333292e-08


In [6]:
print(np.abs(R-C).round(3))

[[0.    0.063 0.029 0.009]
 [0.063 0.    0.036 0.011]
 [0.029 0.036 0.    0.005]
 [0.009 0.011 0.005 0.   ]]


Looks Ok.

# Source

In [7]:
??luriegold

Signature: luriegold(R:numpy.ndarray) -> (<class 'numpy.ndarray'>, <class 'numpy.ndarray'>, <class 'dict'>)
Source:   
def luriegold(R: np.ndarray) -> (np.ndarray, np.ndarray, dict):
    """Lurie-Goldberg Algorithm to adjust a correlation
    matrix to be semipositive definite

    Philip M. Lurie and Matthew S. Goldberg (1998), An Approximate Method
       for Sampling Correlated Random Variables from Partially-Specified
       Distributions, Management Science, Vol 44, No. 2, February 1998, pp
       203-218, URL: http://www.jstor.org/stable/2634496
    """

    # subfunctions
    def xtotril(x: np.ndarray,
                idx: np.ndarray,
                mat: np.ndarray) -> np.ndarray:
        """Create 'L' lower triangular matrix."""
        mat[idx] = x
        return mat

    def xtocorr(x: np.ndarray,
                idx: np.ndarray,
                mat: np.ndarray) -> (np.ndarray, np.ndarray):
        L = xtotril(x, idx, mat)
        C = np.dot(L, L.T)
        return C, L

    